# **<font style="color:black">Base example RNN net</font>**

## **<font style="color:blue">Installation and Imports libraries</font>**

In [1]:
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perfrom on our dataset
from torch.utils.data import (
    DataLoader,
) # Gives easier dataset managment by creating mini batches etc.

from tqdm import tqdm # For a nice progree bar

## **<font style="color:blue">Settings for train</font>**

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 60

## **<font style="color:blue">Create RNN model</font>**

In [15]:
# Create RNN Network
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

## **<font style="color:blue">Load Data</font>**

In [16]:
!mkdir /kaggle/working/dataset/

mkdir: cannot create directory ‘/kaggle/working/dataset/’: File exists


In [17]:
train_dataset = datasets.MNIST(root=os.path.join('/kaggle','working','dataset'), train=True, transform=transforms.ToTensor(), download=True)

In [18]:
test_dataset = datasets.MNIST(root=os.path.join('/kaggle','working','dataset'), train=False, transform=transforms.ToTensor(), download=True)

In [19]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## **<font style="color:blue">Initialize RNN network</font>**

In [20]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

## **<font style="color:blue">Loss and Optimizer</font>**

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## **<font style="color:blue">Train Network</font>**

In [22]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

100%|██████████| 938/938 [00:07<00:00, 128.21it/s]


## **<font style="color:blue">Check accuracy on training & test to see how good our model</font>**

In [23]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [24]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 97.08
Accuracy on test set: 96.58
